In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from analysis import extract_gazes_to_partner
import portion as P
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from analysis import butter_lowpass_filter


## Configuration

- Only retain segments where a complete 30-second conversation occurred.
- Exclude pair 0-3 due to the implementation errors
- Exclude pair 7 since their avatar connection was lost midway, and only the voice remained.

In [2]:
folder_path = '../Data'

conversation_start_times = [
    0,0,0,0,0,   # 0,   1,  2,  3,  4
    30,0,0,0,0,  # 5,   6,  7,  8,  9
    0,30,0,0,0,  # 10, 11, 12, 13, 14
    0,30,0,0,0,  # 15, 16, 17, 18, 19
    0,0,0,0,0,   # 20, 21, 22, 23, 24
    0,0,0,0,0,   # 25, 26, 27, 28, 29
    0,           # 30
]

conversation_end_times = [
    600,600,600,600,600,  # 0,   1,  2,  3,  4
    600,600,600,600,600,  # 5,   6,  7,  8,  9
    600,630,600,600,630,  # 10, 11, 12, 13, 14
    600,630,600,600,600,  # 15, 16, 17, 18, 19
    630,480,630,600,600,  # 20, 21, 22, 23, 24
    600,600,600,630,600,  # 25, 26, 27, 28, 29
    600,                  # 30
]

skips = [
    7,
]


## Process

In [3]:
def process_gaze(raw, t, draw_figure=False):

    adjacent = 0.5/np.tan(np.deg2rad(15))

    result = pd.DataFrame(dict(
        t = raw['t'].values,
        leftanglehead = np.rad2deg(np.arctan2(raw['EYE.LeftEye.Head.distance'], adjacent)),
        leftanglebody = np.rad2deg(np.arctan2(raw['EYE.LeftEye.Body.distance'], adjacent)),
        rightanglehead = np.rad2deg(np.arctan2(raw['EYE.RightEye.Head.distance'], adjacent)),
        rightanglebody = np.rad2deg(np.arctan2(raw['EYE.RightEye.Body.distance'], adjacent)),
        leftconfidence = raw['EYE.LeftEye.Head.confidence.x'].values,
        rightconfidence = raw['EYE.RightEye.Head.confidence.x'].values,
        leftclosed = raw['FACE.EyesClosedL'].values,
        rightclosed = raw['FACE.EyesClosedR'].values,
    ))

    result = result.dropna().reset_index(drop=True)

    if draw_figure:
        plt.subplot(2,1,1)
        plt.plot(result['t'], result['leftangle'])
        
        plt.subplot(2,1,2)
        plt.plot(result['t'], result['rightangle'])


    # low-pass sampling
    sr = len(result)/(result['t'][len(result)-1] - result['t'][0])
    order = 2

    result['leftangle_head'] = butter_lowpass_filter(result['leftanglehead'], 15, sr, order)
    result['leftangle_body'] = butter_lowpass_filter(result['leftanglebody'], 15, sr, order)
    result['rightangle_head'] = butter_lowpass_filter(result['rightanglehead'], 15, sr, order)
    result['rightangle_body'] = butter_lowpass_filter(result['rightanglebody'], 15, sr, order)
    result['leftconfidence'] = butter_lowpass_filter(result['leftconfidence'], 15, sr, order)
    result['rightconfidence'] = butter_lowpass_filter(result['rightconfidence'], 15, sr, order)
    result['leftclosed'] = butter_lowpass_filter(result['leftclosed'], 15, sr, order)
    result['rightclosed'] = butter_lowpass_filter(result['rightclosed'], 15, sr, order)

    if draw_figure:
        plt.subplot(2,1,1)
        plt.plot(result['t'], result['leftangle'], ls='dashed')
        plt.xlim(100,120)
        
        plt.subplot(2,1,2)
        plt.plot(result['t'], result['rightangle'], ls='dashed')
        plt.xlim(100,120)
        
    # blink interpolation
    
    x = result['t'].copy()
    x[result['leftclosed']>0.9] = np.nan
    y = result['leftangle_head'].copy()
    y[result['leftclosed']>0.9] = np.nan
    x, y = x.dropna(), y.dropna()
    f = interpolate.interp1d(x, y, bounds_error=False, kind='cubic', fill_value="extrapolate")
    result['leftangle_head'] = f(result['t'])

    x = result['t'].copy()
    x[result['rightclosed']>0.9] = np.nan
    y = result['rightangle_head'].copy()
    y[result['rightclosed']>0.9] = np.nan
    x, y = x.dropna(), y.dropna()
    f = interpolate.interp1d(x, y, bounds_error=False, kind='cubic', fill_value="extrapolate")
    result['rightangle_head'] = f(result['t'])

    x = result['t'].copy()
    x[result['leftclosed']>0.9] = np.nan
    y = result['leftangle_body'].copy()
    y[result['leftclosed']>0.9] = np.nan
    x, y = x.dropna(), y.dropna()
    f = interpolate.interp1d(x, y, bounds_error=False, kind='cubic', fill_value="extrapolate")
    result['leftangle_body'] = f(result['t'])

    x = result['t'].copy()
    x[result['rightclosed']>0.9] = np.nan
    y = result['rightangle_body'].copy()
    y[result['rightclosed']>0.9] = np.nan
    x, y = x.dropna(), y.dropna()
    f = interpolate.interp1d(x, y, bounds_error=False, kind='cubic', fill_value="extrapolate")
    result['rightangle_body'] = f(result['t'])

    result['head'] = 0
    result.loc[result['rightangle_head'] < 1.652, 'head'] = 1

    result['body'] = 0
    result.loc[result['rightangle_body'] < 1.652, 'body'] = 1

    result['all'] = 0
    result.loc[(result['rightangle_head'] < 1.652)|(result['rightangle_body'] < 1) , 'all'] = 1

    new_result = pd.DataFrame(dict(
        t = interpolate.interp1d(result.t, result.t, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        leftanglehead = interpolate.interp1d(result.t, result.leftanglehead, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        leftanglebody = interpolate.interp1d(result.t, result.leftanglebody, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        leftconfidence = interpolate.interp1d(result.t, result.leftconfidence, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        leftclosed = interpolate.interp1d(result.t, result.leftclosed, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        rightanglehead = interpolate.interp1d(result.t, result.rightanglehead, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        rightanglebody = interpolate.interp1d(result.t, result.rightanglebody, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        rightconfidence = interpolate.interp1d(result.t, result.rightconfidence, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        rightclosed = interpolate.interp1d(result.t, result.rightclosed, bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        head = interpolate.interp1d(result.t, result['head'], bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        body = interpolate.interp1d(result.t, result['body'], bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
        all = interpolate.interp1d(result.t, result['all'], bounds_error=False, kind='nearest', fill_value="extrapolate")(t),
    ))


    return new_result

In [4]:
def interval2df(interval):
    df = pd.DataFrame([[i.lower, i.upper, i.upper-i.lower] for i in interval], columns=['start', 'end', 'duration'])
    return df

In [5]:
def compute_interval_length(interval):
    df = pd.DataFrame([[i.lower, i.upper, i.upper-i.lower] for i in interval], columns=['start', 'end', 'duration'])
    return df['duration'].sum()

In [6]:
def get_interval_count(interval):
    df = pd.DataFrame([[i.lower, i.upper, i.upper-i.lower] for i in interval], columns=['start', 'end', 'duration'])
    return len(df)

In [7]:
def compute_gaps(turns):
    _gaps = []
    if len(turns) > 2:
        for i in range(1, len(turns)):
            end = max(turns['start'][i], turns['end'][i-1])
            start = min(turns['start'][i], turns['end'][i-1])
            _gaps.append(dict(start=start,  end=end, duration=end-start))
    else:
        _gaps.append(dict(start=np.nan,  end=np.nan, duration=np.nan))
    return pd.DataFrame(_gaps)

In [8]:
def inspect_intersections_per_turn(turns, gaze_interval):
    turns = turns.copy()
    turns['gaze'] = 0
    turns['gazecount'] = 0
    turns['gazeduration'] = 0.0

    for i in range(len(turns)):
        turn = turns.iloc[i,:]
        for j in range(len(gaze_interval)):
            interval = gaze_interval.iloc[j,:]
            if ((interval.start >= turn.start)&(interval.start < turn.end)) \
                 | ((interval.end >= turn.start)&(interval.end < turn.end)):
                turns.loc[i, 'gaze'] = 1
                turns.loc[i, 'gazecount'] = turns.loc[i, 'gazecount'] + 1
                turns.loc[i, 'gazeduration'] = turns.loc[i, 'gazeduration'] + (min(interval.end, turn.end) - max(interval.start, turn.start))

    return turns

Main process

In [11]:
# generate turns.csv with analysis_voice.ipynb
turns_all = pd.read_csv("processed/turns.csv")
bins_all = pd.DataFrame()

df = pd.DataFrame([])


for i in range(4,31):

    if i in set(skips): 
        continue
    
    for p in ['a', 'b']:
        
        pid = f'{i}{p}'
        print(f'# {pid} processing')
        starttime = conversation_start_times[i]
        endtime = conversation_end_times[i]

        # load social connection responses
        voice_start_time = pd.read_csv(f'{folder_path}/RawData/{pid}_video_timestamp.csv', names=['idx', 'ts', 'freq'], header=None)['ts'][0]
        social_connection = pd.read_table(f'{folder_path}/RawData/{pid}_video_measure.tsv')

        # load turns
        turns = turns_all[turns_all.pid == pid].reset_index(drop=True)
        self_turns = turns[(turns.speaker=='self')].copy().reset_index(drop=True)
        partner_turns = turns[(turns.speaker=='partner')].copy().reset_index(drop=True)

        # load gazes
        raw_pose_self = pd.read_table(f'{folder_path}/RawData/{pid}_self.tsv')
        raw_pose_partner = pd.read_table(f'{folder_path}/RawData/{pid}_remote.tsv')

        # synchronization
        raw_pose_self['timestamp'] = raw_pose_self['POSE.timestamp']
        if pid in ['20a', '21a']:
            # due to timestamp issue for 21a, 22a...
            raw_pose_self['t'] = (raw_pose_self['timestamp'] * 0.001 - (raw_pose_self['timestamp'][0] * 0.001 - 0.003)) # avarage offset 0.003(sd=0.030)
            raw_pose_partner['t'] = (raw_pose_partner['timestamp'] * 0.001 - (raw_pose_self['timestamp'][0] * 0.001 - 0.003))
        else: 
            raw_pose_self['t'] = (raw_pose_self['timestamp'] * 0.001 - voice_start_time)
            raw_pose_partner['t'] = (raw_pose_partner['timestamp'] * 0.001 - voice_start_time)

        gaze_self = process_gaze(raw_pose_self, np.arange(starttime, endtime, 1/30))
        gaze_partner = process_gaze(raw_pose_partner, np.arange(starttime, endtime, 1/30))

        # extract recognizable partner's gaze
        # gaze_partner['head_recognizable'] = 0
        # gaze_partner.loc[((gaze_self.rightanglehead < 9)&(gaze_partner['head']>0.5)) ,'head_recognizable'] = 1
        head_gaze_self = extract_gazes_to_partner(gaze_self['t'], gaze_self['head'])
        head_gaze_self_interval = P.Interval(*[P.closed(head_gaze_self.loc[i, 'start'], head_gaze_self.loc[i, 'end']) for i in range(len(head_gaze_self))])
        head_gaze_partner = extract_gazes_to_partner(gaze_partner['t'], gaze_partner['head'])
        head_gaze_partner_interval = P.Interval(*[P.closed(head_gaze_partner.loc[i, 'start'], head_gaze_partner.loc[i, 'end']) for i in range(len(head_gaze_partner))])
        head_gaze_mutual_interval = head_gaze_self_interval.intersection(head_gaze_partner_interval)
        head_gaze_mutual = interval2df(head_gaze_mutual_interval)

        # check errors for gaze2head
        # plt.figure(figsize=(8,2), dpi=180)
        # plt.title(pid)
        # plt.subplot(1,2,1)
        # plt.hist(gaze_self.rightanglehead, bins=np.arange(-3, 15, 0.1))
        # plt.xlabel('degree')
        # plt.grid()
        # plt.subplot(1,2,2)
        # plt.hist(gaze_partner.rightanglehead, bins=np.arange(-3, 15, 0.1))
        # plt.grid()
        # plt.xlabel('degree')
        # plt.tight_layout()
        # plt.show()


        # append segment data
        _bins = []  # List to store the rows of the resulting DataFrame
        start, end = int(starttime) // 30, int(endtime) // 30  # Convert start and end time to segment index

        for j in range(start, end):
            t_seg_start, t_seg_end = j * 30, (j + 1) * 30

            # rating
            rating_seg = social_connection[(social_connection.TimeStamp>= t_seg_start) 
                & (social_connection.TimeStamp < t_seg_end)]['SocialConnection']

            # turn
            turns_seg = turns[(turns.start >= t_seg_start) & (turns.start < t_seg_end)].copy().reset_index(drop=True)
            self_turns_seg = turns_seg[(turns_seg.speaker=='self')].copy().reset_index(drop=True)
            partner_turns_seg = turns_seg[(turns_seg.speaker=='partner')].copy().reset_index(drop=True)
            gaps_seg = compute_gaps(turns_seg)

            # gaze
            mutual_gaze_seg = head_gaze_mutual[(head_gaze_mutual.end >= t_seg_start) & (head_gaze_mutual.start < t_seg_end)].copy().reset_index(drop=True)
            self_gaze_seg = head_gaze_self[(head_gaze_self.end >= t_seg_start) & (head_gaze_self.start < t_seg_end)].copy().reset_index(drop=True)
            partner_gaze_seg = head_gaze_partner[(head_gaze_partner.end >= t_seg_start) & (head_gaze_partner.start < t_seg_end)].copy().reset_index(drop=True)
            
            # crop gazes
            for k in range(len(mutual_gaze_seg)):
                mutual_gaze_seg.loc[k, 'end'] = min(t_seg_end, mutual_gaze_seg.loc[k, 'end'])
                mutual_gaze_seg.loc[k, 'start'] = max(t_seg_start, mutual_gaze_seg.loc[k, 'start'])
                mutual_gaze_seg.loc[k, 'duration'] = mutual_gaze_seg.loc[k, 'end'] - mutual_gaze_seg.loc[k, 'start']

            for k in range(len(self_gaze_seg)):
                self_gaze_seg.loc[k, 'end'] = min(t_seg_end, self_gaze_seg.loc[k, 'end'])
                self_gaze_seg.loc[k, 'start'] = max(t_seg_start, self_gaze_seg.loc[k, 'start'])
                self_gaze_seg.loc[k, 'duration'] = self_gaze_seg.loc[k, 'end'] - self_gaze_seg.loc[k, 'start']

            for k in range(len(partner_gaze_seg)):
                partner_gaze_seg.loc[k, 'end'] = min(t_seg_end, partner_gaze_seg.loc[k, 'end'])
                partner_gaze_seg.loc[k, 'start'] = max(t_seg_start, partner_gaze_seg.loc[k, 'start'])
                partner_gaze_seg.loc[k, 'duration'] = partner_gaze_seg.loc[k, 'end'] - partner_gaze_seg.loc[k, 'start']

            turn_durations = P.Interval(*[P.closed(turns_seg.loc[i, 'start'], turns_seg.loc[i, 'end']) for i in range(len(turns_seg))])
            self_turn_durations = P.Interval(*[P.closed(self_turns_seg.loc[i, 'start'], self_turns_seg.loc[i, 'end']) for i in range(len(self_turns_seg))])
            partner_turn_durations = P.Interval(*[P.closed(partner_turns_seg.loc[i, 'start'], partner_turns_seg.loc[i, 'end']) for i in range(len(partner_turns_seg))])
            gap_durations = P.Interval(*[P.closed(gaps_seg.loc[i, 'start'], gaps_seg.loc[i, 'end']) for i in range(len(gaps_seg))])

            start_turn_durations = P.Interval(*[P.closed(turns_seg.loc[i, 'start'], turns_seg.loc[i, 'start']+1) for i in range(len(turns_seg))])
            start_self_turn_durations = P.Interval(*[P.closed(self_turns_seg.loc[i, 'start'], self_turns_seg.loc[i, 'start']+1) for i in range(len(self_turns_seg))])
            start_partner_turn_durations = P.Interval(*[P.closed(partner_turns_seg.loc[i, 'start'], partner_turns_seg.loc[i, 'start']+1) for i in range(len(partner_turns_seg))])

            end_turn_durations = P.Interval(*[P.closed(turns_seg.loc[i, 'end']-1, turns_seg.loc[i, 'end']) for i in range(len(turns_seg))])
            end_self_turn_durations = P.Interval(*[P.closed(self_turns_seg.loc[i, 'end']-1, self_turns_seg.loc[i, 'end']) for i in range(len(self_turns_seg))])
            end_partner_turn_durations = P.Interval(*[P.closed(partner_turns_seg.loc[i, 'end']-1, partner_turns_seg.loc[i, 'end']) for i in range(len(partner_turns_seg))])

            self_gaze_durations = P.Interval(*[P.closed(self_gaze_seg.loc[i, 'start'], self_gaze_seg.loc[i, 'end']) for i in range(len(self_gaze_seg))])
            partner_gaze_durations = P.Interval(*[P.closed(partner_gaze_seg.loc[i, 'start'], partner_gaze_seg.loc[i, 'end']) for i in range(len(partner_gaze_seg))])
            mutual_gaze_durations = P.Interval(*[P.closed(mutual_gaze_seg.loc[i, 'start'], mutual_gaze_seg.loc[i, 'end']) for i in range(len(mutual_gaze_seg))])

            _bins.append(dict(Pid=pid,
                Session=pid[:-1],
                Segment=j,
                StartTime=t_seg_start,
                EndTime=t_seg_end,

                # quantity
                # self gaze to head
                # partner gaze to head
                # mutual face gaze duration and frequency / waiter frequency / finisher frequency
                SelfGazeDuration=self_gaze_seg['duration'].sum(),
                PartnerGazeDuration=partner_gaze_seg['duration'].sum(),
                EyeContactDuration=mutual_gaze_seg['duration'].sum(),

                TurnSelfGazeDuration = compute_interval_length(turn_durations.intersection(self_gaze_durations)),
                TurnPartnerGazeDuration = compute_interval_length(turn_durations.intersection(partner_gaze_durations)),
                TurnMutualGazeDuration = compute_interval_length(turn_durations.intersection(mutual_gaze_durations)),

                GapSelfGazeDuration = compute_interval_length(gap_durations.intersection(self_gaze_durations)),
                GapPartnerGazeDuration = compute_interval_length(gap_durations.intersection(partner_gaze_durations)),
                GapMutualGazeDuration = compute_interval_length(gap_durations.intersection(mutual_gaze_durations)),

                SelfTurnSelfGazeDuration = compute_interval_length(self_turn_durations.intersection(self_gaze_durations)),
                SelfTurnPaThetnerGazeDuration = compute_interval_length(self_turn_durations.intersection(partner_gaze_durations)),
                SelfTurnMutualGazeDuration = compute_interval_length(self_turn_durations.intersection(mutual_gaze_durations)),
                
                PartnerTurnSelfGazeDuration = compute_interval_length(partner_turn_durations.intersection(self_gaze_durations)),
                PartnerTurnPartnerGazeDuration = compute_interval_length(partner_turn_durations.intersection(partner_gaze_durations)),
                PartnerTurnMutualGazeDuration = compute_interval_length(partner_turn_durations.intersection(mutual_gaze_durations)),

                SocialConnection=rating_seg.mean(),
                SocialConnectionStd=rating_seg.std()
            ))

        bins_all = pd.concat([bins_all, pd.DataFrame(_bins)]).reset_index(drop=True)

# 4a processing
# 4b processing
# 5a processing
# 5b processing
# 6a processing
# 6b processing
# 8a processing
# 8b processing
# 9a processing
# 9b processing
# 10a processing
# 10b processing
# 11a processing
# 11b processing
# 12a processing
# 12b processing
# 13a processing
# 13b processing
# 14a processing
# 14b processing
# 15a processing
# 15b processing
# 16a processing
# 16b processing
# 17a processing
# 17b processing
# 18a processing
# 18b processing
# 19a processing
# 19b processing
# 20a processing
# 20b processing
# 21a processing
# 21b processing
# 22a processing
# 22b processing
# 23a processing
# 23b processing
# 24a processing
# 24b processing
# 25a processing
# 25b processing
# 26a processing
# 26b processing
# 27a processing
# 27b processing
# 28a processing
# 28b processing
# 29a processing
# 29b processing
# 30a processing
# 30b processing


In [12]:
bins_all.to_csv("processed/gazes_within.csv")
bins_all


,Pid,Session,Segment,StartTime,EndTime,SelfGazeDuration,PartnerGazeDuration,EyeContactDuration,TurnSelfGazeDuration,TurnPartnerGazeDuration,...,GapPartnerGazeDuration,GapMutualGazeDuration,SelfTurnSelfGazeDuration,SelfTurnPaThetnerGazeDuration,SelfTurnMutualGazeDuration,PartnerTurnSelfGazeDuration,PartnerTurnPartnerGazeDuration,PartnerTurnMutualGazeDuration,SocialConnection,SocialConnectionStd
0,4a,4,0,0,30,12.949999,3.911000,1.1160,7.4840,2.836300,...,0.0000,0.0000,4.7753,2.2300,0.6160,2.8087,0.6063,0.1390,20.79160,3.553701e-15
1,4a,4,1,30,60,11.122300,5.303000,0.9700,7.6004,4.018300,...,1.2847,0.0410,7.2840,3.6433,0.9290,0.4964,0.3750,0.0000,24.25687,3.553701e-15
2,4a,4,2,60,90,14.186700,0.887000,0.4450,9.5843,0.458000,...,0.3610,0.2920,4.4716,0.2500,0.0850,5.6127,0.2080,0.0000,30.32109,7.107402e-15
3,4a,4,3,90,120,10.371000,13.294300,3.8350,10.1730,11.953600,...,1.4877,0.3890,5.3117,7.9750,2.7457,5.5506,4.2786,1.3003,21.70032,0.000000e+00
4,4a,4,4,120,150,5.512000,17.680000,2.8180,3.9457,11.890600,...,2.9264,0.6723,1.2637,9.8670,1.0250,2.6820,2.1763,0.4450,25.02119,3.553701e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,30b,30,15,450,480,16.360600,18.371999,9.2430,13.9910,16.677199,...,2.1208,0.7996,11.0884,12.0194,5.7128,3.4402,5.6302,3.2042,39.89297,7.107402e-15
1034,30b,30,16,480,510,20.583400,12.932400,8.3334,15.0358,10.089600,...,2.2554,1.3536,3.5314,2.7436,1.7160,11.7244,7.4976,5.3010,38.16034,7.107402e-15
1035,30b,30,17,510,540,26.537000,12.409600,9.7786,19.7314,9.967800,...,2.6994,1.3928,11.6514,5.9812,5.0756,9.1000,4.6842,4.1466,37.43840,7.107402e-15
1036,30b,30,18,540,570,27.474000,14.333400,12.0574,18.6068,10.397000,...,2.8910,1.9282,6.5914,5.0236,4.0550,12.0154,5.3734,5.0288,37.43840,7.107402e-15


In [13]:
bins_across = bins_all.groupby(by='Pid', as_index=False).mean(numeric_only=True)
bins_across['pid'] = bins_across['Pid']
bins_across.to_csv("processed/gazes_across.csv")

# Descriptive Statistics

In [14]:
bins_all.describe()

,Segment,StartTime,EndTime,SelfGazeDuration,PartnerGazeDuration,EyeContactDuration,TurnSelfGazeDuration,TurnPartnerGazeDuration,TurnMutualGazeDuration,GapSelfGazeDuration,GapPartnerGazeDuration,GapMutualGazeDuration,SelfTurnSelfGazeDuration,SelfTurnPaThetnerGazeDuration,SelfTurnMutualGazeDuration,PartnerTurnSelfGazeDuration,PartnerTurnPartnerGazeDuration,PartnerTurnMutualGazeDuration,SocialConnection,SocialConnectionStd
count,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1038.000000,1028.000000,1.028000e+03
mean,9.614644,288.439306,318.439306,18.156231,18.149119,11.002662,13.645864,13.744214,8.334741,2.819740,2.780903,1.707231,5.472219,8.197505,4.151037,8.481530,5.880604,4.387083,55.037214,9.303838e-15
std,5.795108,173.853239,173.853239,7.170339,7.174420,6.271748,6.201598,6.273491,5.106447,1.989459,1.988560,1.567521,3.906245,5.670463,3.531130,5.678251,3.936941,3.563982,23.410854,8.837067e-15
min,0.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000e+00
25%,5.000000,150.000000,180.000000,13.530900,13.519125,6.292775,9.384450,9.520100,4.310950,1.306750,1.271000,0.478800,2.488425,3.767200,1.259025,4.043300,2.870450,1.437475,37.438400,0.000000e+00
50%,10.000000,300.000000,330.000000,19.162300,19.174500,10.828000,14.151100,14.369400,8.351299,2.537500,2.477250,1.359000,4.840450,7.404950,3.357700,7.708650,5.515500,3.735000,59.962640,7.107402e-15
75%,15.000000,450.000000,480.000000,23.887701,23.900000,15.080250,18.227825,18.424750,11.855275,3.955550,3.924500,2.497875,7.876025,11.892700,6.122750,12.088175,8.149425,6.452425,74.658240,1.421480e-14
max,20.000000,600.000000,630.000000,30.000000,30.000000,28.755900,28.840000,27.961600,23.498300,11.068000,10.693000,10.666000,21.544600,26.980200,18.651000,27.580000,20.159000,18.273800,100.000000,4.264441e-14


In [15]:
bins_all.SelfTurnSelfGazeDuration.describe()

count    1038.000000
mean        5.472219
std         3.906245
min         0.000000
25%         2.488425
50%         4.840450
75%         7.876025
max        21.544600
Name: SelfTurnSelfGazeDuration, dtype: float64

In [16]:
bins_all.SelfTurnMutualGazeDuration.describe()

count    1038.000000
mean        4.151037
std         3.531130
min         0.000000
25%         1.259025
50%         3.357700
75%         6.122750
max        18.651000
Name: SelfTurnMutualGazeDuration, dtype: float64

In [17]:
bins_all.GapSelfGazeDuration.describe()


count    1038.000000
mean        2.819740
std         1.989459
min         0.000000
25%         1.306750
50%         2.537500
75%         3.955550
max        11.068000
Name: GapSelfGazeDuration, dtype: float64

In [18]:
bins_all.PartnerTurnSelfGazeDuration.describe()

count    1038.000000
mean        8.481530
std         5.678251
min         0.000000
25%         4.043300
50%         7.708650
75%        12.088175
max        27.580000
Name: PartnerTurnSelfGazeDuration, dtype: float64

In [19]:
bins_all.PartnerTurnMutualGazeDuration.describe()

count    1038.000000
mean        4.387083
std         3.563982
min         0.000000
25%         1.437475
50%         3.735000
75%         6.452425
max        18.273800
Name: PartnerTurnMutualGazeDuration, dtype: float64

In [20]:
bins_all.GapSelfGazeDuration.describe()

count    1038.000000
mean        2.819740
std         1.989459
min         0.000000
25%         1.306750
50%         2.537500
75%         3.955550
max        11.068000
Name: GapSelfGazeDuration, dtype: float64

In [21]:
bins_all.GapMutualGazeDuration.describe()

count    1038.000000
mean        1.707231
std         1.567521
min         0.000000
25%         0.478800
50%         1.359000
75%         2.497875
max        10.666000
Name: GapMutualGazeDuration, dtype: float64